In [1]:
## 使用Qwen-turbo，对保险客户数据表进行SQL查询

In [2]:
import json
import os
import dashscope
from dashscope.api_entities.dashscope_response import Role
import time
import pandas as pd
import re
# 从环境变量获取 dashscope 的 API Key
api_key = os.environ.get('DASHSCOPE_API_KEY')
dashscope.api_key = api_key

# 封装模型响应函数
def get_response(messages):
    response = dashscope.Generation.call(
        model='qwen-turbo-latest',
        messages=messages,
        result_format='message'  # 将输出设置为message形式
    )
    return response

# 从模型响应中提取SQL代码
def get_sql_code(response):
    # 查找```sql和```之间的内容
    pattern = r'```sql(.*?)```'
    match = re.search(pattern, response.output.choices[0].message.content, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        # 如果没有找到```sql标记，尝试查找任何```之间的内容
        pattern = r'```(.*?)```'
        match = re.search(pattern, response.output.choices[0].message.content, re.DOTALL)
        if match:
            return match.group(1).strip()
        else:
            # 如果没有找到任何代码块，返回整个响应
            return response.output.choices[0].message.content

# 得到sql
def get_sql(query, table_description):
    start_time = time.time()
    sys_prompt = """我正在编写SQL，以下是数据库中的数据表和字段，请思考：哪些数据表和字段是该SQL需要的，然后编写对应的SQL，如果有多个查询语句，请尝试合并为一个。编写SQL请采用```sql
    """
    user_prompt = f"""{table_description}
=====
我要写的SQL是：{query}
请思考：哪些数据表和字段是该SQL需要的，然后编写对应的SQL
"""
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    response = get_response(messages)
    return response

########## 以下参数需要进行确认 ##########
save_file = f'sql_result_qwen_turbo.xlsx'
data_file = './data/数据表字段说明-精简1.txt'
qa_file = './qa_list-2.txt'

In [3]:
# 读取 数据表字段说明-精简
with open(data_file, 'r', encoding='utf-8') as file:
    table_description = file.read()
# 读取 SQL问题列表
with open(qa_file, 'r', encoding='utf-8') as file:
    qa_list = file.read()
qa_list = qa_list.split('=====')

# 保存SQL结果
sql_list = []
markdown_list = []
time_list = []
for qa in qa_list:
    query = qa
    query = query.replace('\n', '')
    print(query)
    start_time = time.time()
    # 请求生成sql
    response = get_sql(query, table_description)
    use_time = round(time.time()-start_time, 2)
    time_list.append(use_time)
    print('SQL生成时间：', use_time)
    print('response=', response.output.choices[0].message.content)
    # 提取生成的SQL
    sql = get_sql_code(response)
    print('SQL: {}'.format(sql))
    sql_list.append(sql)

获取所有客户的姓名和联系电话。
SQL生成时间： 2.21
response= 根据你的需求“获取所有客户的姓名和联系电话”，需要的**数据表和字段**如下：

### 需要的数据表：
- **CustomerInfo**（客户信息表）

### 需要的字段：
- **Name**（客户姓名）
- **PhoneNumber**（联系电话）

---

### 编写的 SQL 语句如下：

```sql
SELECT Name, PhoneNumber
FROM CustomerInfo;
```

该查询从 `CustomerInfo` 表中提取所有客户的姓名和联系电话。没有其他表或字段涉及，因此无需合并多个查询。
SQL: SELECT Name, PhoneNumber
FROM CustomerInfo;
找出所有已婚客户的客户ID和配偶姓名。
SQL生成时间： 29.79
response= 根据你的需求“找出所有已婚客户的客户ID和配偶姓名”，我们可以分析如下：

---

### ✅ **需要的数据表和字段**

#### 1. **CustomerInfo 表**
- `CustomerID`：客户ID
- `MaritalStatus`：婚姻状况（用于筛选“已婚”客户）
- `Name`：客户姓名（可能与配偶姓名相关，但通常不是直接存储配偶姓名）

#### 2. **BeneficiaryInfo 表**
- `BeneficiaryID`：受益人ID（用于关联客户和其配偶）
- `Name`：受益人姓名（即配偶的姓名）
- `Relationship`：受益人关系（如“配偶”）

> 注意：**客户信息表中并没有直接存储配偶姓名**，而配偶作为受益人，一般会记录在 **BeneficiaryInfo 表** 中。因此，我们需要通过 **CustomerID 和 BeneficiaryID 的关联** 来获取配偶姓名。

---

### ✅ **SQL 查询逻辑**
- 从 `CustomerInfo` 中筛选出 `MaritalStatus = '已婚'` 的客户。
- 通过 `CustomerID` 与 `BeneficiaryInfo` 表中的 `CustomerID` 关联。
- 筛选 `BeneficiaryInfo` 中 `Rela

In [4]:
result = pd.DataFrame(columns=['QA', 'SQL', 'time'])
result['QA'] = qa_list
result['SQL'] = sql_list
result['time'] = time_list
result.to_excel(save_file, index=False)
result

,QA,SQL,time
0,获取所有客户的姓名和联系电话。\n,"SELECT Name, PhoneNumber\nFROM CustomerInfo;",2.21
1,\n找出所有已婚客户的客户ID和配偶姓名。\n,"SELECT \n c.CustomerID,\n b.Name AS Spou...",29.79
2,\n查询所有未支付保费的保单号和客户姓名。\n,"SELECT \n p.PolicyNumber,\n c.Name AS Cu...",5.33
3,\n找出所有理赔金额大于10000元的理赔记录，并列出相关客户的姓名和联系电话。\n,"SELECT \n c.Name AS 客户姓名,\n c.PhoneNumbe...",7.98
4,\n查找代理人的姓名和执照到期日期，按照执照到期日期升序排序。\n,"SELECT Name, LicenseExpirationDate\nFROM Agent...",2.74
5,\n获取所有保险产品的产品名称和保费，按照保费降序排序。\n,"SELECT ProductName, Premium\nFROM ProductInfo\...",3.17
6,\n查询销售区域在上海的员工，列出他们的姓名和职位。\n,"SELECT Name, Position\nFROM EmployeeInfo\nWHER...",3.28
7,\n找出所有年龄在30岁以下的客户，并列出其客户ID、姓名和出生日期。\n,"SELECT CustomerID, Name, DateOfBirth\nFROM Cus...",7.35
8,\n查找所有已审核但尚未支付的理赔记录，包括理赔号、审核人和审核日期。\n,"SELECT \n ClaimNumber,\n ClaimHandler,\n...",6.00
9,\n获取每个产品类型下的平均保费，以及该产品类型下的产品数量。,"SELECT \n ProductType,\n AVG(Premium) AS...",6.45
